# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [40]:
# load file
file = "output_data/cities.csv"

# convert to data frame
cities_pd = pd.DataFrame(pd.read_csv(file))

# --- display the dataframe ---
cities_pd


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lompoc,34.6391,-120.4579,63.09,69,0,6.91,US,1643734033
1,1,namibe,-15.1961,12.1522,75.92,64,22,13.04,AO,1643734033
2,2,cape town,-33.9258,18.4232,80.89,52,25,1.99,ZA,1643733776
3,3,mar del plata,-38.0023,-57.5575,78.87,51,0,24.00,AR,1643733885
4,4,coihaique,-45.5752,-72.0662,67.95,45,75,9.22,CL,1643733757
...,...,...,...,...,...,...,...,...,...,...
540,540,san vicente,-26.6167,-54.1333,96.73,13,0,3.60,AR,1643734860
541,541,neiafu,-18.6500,-173.9833,80.04,72,18,6.58,TO,1643734860
542,542,umm lajj,25.0213,37.2685,70.32,31,0,15.48,SA,1643734861
543,543,stokmarknes,68.5646,14.9108,30.16,46,0,14.97,NO,1643734706


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [41]:
# access gmaps
gmaps.configure(api_key=g_key)

# Lat and Lng as locations and Humidity as the weight 
locations = cities_pd[["Lat", "Lng"]]
weights = cities_pd["Humidity"]



In [42]:
# --- customize the size of the figure and add heatmap layer to map ---
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False)
fig.add_layer(heatmap_layer)

# --- set maximum intensity and point_radius settings ---
heatmap_layer.max_intensity = max(weights)
heatmap_layer.point_radius = 3

# --- display heatmap ---
fig



Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
# --- create dataframe with conditional statements to fit the below criteria ---
# 1. A max temperature lower than 80 degrees but higher than 70.
# 2. Wind speed less than 10 mph.
# 3. Zero cloudiness.

ideal_pd = cities_pd.loc[(cities_pd["Max Temp"] > 70) & 
                         (cities_pd["Max Temp"] < 80) & 
                         (cities_pd["Wind Speed"] < 10) & 
                         (cities_pd["Cloudiness"] == 0)]

ideal_pd = ideal_pd.reset_index(drop=True).dropna()
ideal_pd

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,30,chuy,-33.6971,-53.4616,75.79,38,0,5.46,UY,1643734043
1,35,cabo san lucas,22.8909,-109.9124,72.52,49,0,2.17,MX,1643734045
2,166,rocha,-34.4833,-54.3333,78.19,34,0,2.98,UY,1643734027
3,306,jacksonville,30.3322,-81.6556,70.11,56,0,1.99,US,1643734348
4,350,sur,22.5667,59.5289,74.34,68,0,6.46,OM,1643734564
5,401,paso de los toros,-32.8167,-56.5167,79.14,31,0,3.80,UY,1643734639
6,461,brandon,27.9378,-82.2859,72.30,58,0,1.99,US,1643734608
7,516,arlit,18.7369,7.3853,70.74,8,0,5.06,NE,1643734853


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
# store into variable named hotel_df 
hotel_df = ideal_pd[["City", "Country", "Lat", "Lng"]].copy()

# Add a "Hotel Name" column to the DataFrame with null values 
hotel_df["Hotel Name"] = " "
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,chuy,UY,-33.6971,-53.4616,
1,cabo san lucas,MX,22.8909,-109.9124,
2,rocha,UY,-34.4833,-54.3333,
3,jacksonville,US,30.3322,-81.6556,
4,sur,OM,22.5667,59.5289,
5,paso de los toros,UY,-32.8167,-56.5167,
6,brandon,US,27.9378,-82.2859,
7,arlit,NE,18.7369,7.3853,


In [34]:
# query Google Places API 
query_places = "hotel"
query_radius = 5000
query_type = "lodging"

# set up a parameters dictionary 
params = {
    "keyword": query_places,
    "radius": query_radius,
    "type": query_type,
    "key": g_key
}

# define base url 
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [46]:
# make an API call for each city in the data frame

for index, row in hotel_df.iterrows():
    
    # --- get the coordinates and store in params dictionary ---
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # --- call the API and get hotel details for each city ---
    hotel_info = requests.get(base_url, params).json()
    
    # --- use exception handling to save the first hotel name for each city ---
    
    try:
        hotel_name = hotel_info["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"City '{row['City']}' The nearest hotel is {hotel_name}")
        print("-------------------------------------------------------------------------")
              
    except:
        print(f"Could not find information for city {row['City']}")
        print("...............................skipping..................................")    
        print("-------------------------------------------------------------------------")
        

City 'chuy' The nearest hotel is Hotel Internacional
-------------------------------------------------------------------------
City 'cabo san lucas' The nearest hotel is Medano Hotel and Suites
-------------------------------------------------------------------------
City 'rocha' The nearest hotel is Hotel Trocadero
-------------------------------------------------------------------------
City 'jacksonville' The nearest hotel is Marriott Jacksonville Downtown
-------------------------------------------------------------------------
City 'sur' The nearest hotel is Sur Plaza Hotel
-------------------------------------------------------------------------
City 'paso de los toros' The nearest hotel is Midland Hotel
-------------------------------------------------------------------------
City 'brandon' The nearest hotel is Country Inn & Suites by Radisson, Tampa/Brandon, FL
-------------------------------------------------------------------------
City 'arlit' The nearest hotel is Hôtel Teme

In [36]:
# remove rows with null values
hotel_df.dropna(how="any",inplace=True)
hotel_df.reset_index(drop=True, inplace=True)

hotel_df

,City,Country,Lat,Lng,Hotel Name
0,chuy,UY,-33.6971,-53.4616,Hotel Internacional
1,cabo san lucas,MX,22.8909,-109.9124,Medano Hotel and Suites
2,rocha,UY,-34.4833,-54.3333,Hotel Trocadero
3,jacksonville,US,30.3322,-81.6556,Marriott Jacksonville Downtown
4,sur,OM,22.5667,59.5289,Sur Plaza Hotel
5,paso de los toros,UY,-32.8167,-56.5167,Midland Hotel
6,brandon,US,27.9378,-82.2859,"Country Inn & Suites by Radisson, Tampa/Brando..."
7,arlit,NE,18.7369,7.3853,Hôtel Temet


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…